# 0. Введение
В рамках этой практики, мы разберём состязательные атаки на системы ИИ на примере моделей для классификации изображений. Очевидно, что нейронные сети являются очень мощным инструментом для распознавания закономерностей в данных и, в частности, для выполнения классификации изображений. Однако насколько надежны эти модели на самом деле? Можно ли "обмануть" модель и создать изображения, которые сеть будет классифицировать неправильно?
## Fast Gradient Sign Method (FGSM)
Одной из первых предложенных стратегий для реализации состязательной атаки является метод быстрого градиентного знака (FGSM), разработанный Иэном Гудфеллоу и др. в 2014 году. Получив исходное изображение на вход, алгоритм генерирует состязательное изображение, используя следующую формулу:
$$
    adv\_x = x + \epsilon * sign(∇_{x}J(\Theta, x, y))
$$
Значение $$J(\Theta,x,y)$$ представляет потери сети при классификации входного изображения $x$ как метки $y$ ; $ϵ$ — интенсивность шума, а $adv\_x$ — выходное состязательное изображение. Мы изменяем входное изображение $x$ в направлении максимизации потерь $J(\Theta,x,y)$. Во время обучения обычной нейронной сети, мы стараемся свести функцию потерь к минимуму, а здесь — действуем ровно наоборот.
# 1. Импорт библиотек и датасета
В данном разделе вам необходимо выполнить импорт всех используемых вами функций и библиотек. Необходимый минимум приведён ниже. Запрещено использовать библиотеки типа foolbox и т.д.

In [5]:
!pip install python-mnist
!pip install tensorflow==2.15.1
!pip install keras==2.15.0
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
import numpy as np
import cv2

from keras.layers import Dense
from keras.models import Sequential, load_model
from keras.layers import Flatten
from keras.losses import MSE
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import Dropout

ERROR: Could not find a version that satisfies the requirement tensorflow==2.15.1 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0)
ERROR: No matching distribution found for tensorflow==2.15.1


ImportError: cannot import name 'MSE' from 'keras.losses' (C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\api\losses\__init__.py)

# 2. Создание CNN (не телеканал)
В данном разделе нам необходимо создать простую CNN (Convolutional Neural Network) для классификации изображений из набора данных MNIST

Для этого сперва нужно инициализировать модель типа Sequential (простую полносвязную сеть), а затем добавить к ней следующие слои:

Первый набор слоёв Conv2D => RELU => BatchNormalization
Второй набор слоёв Conv2D => RELU => BatchNormalization
Набор слоёв FC => RELU
Flatten => Dense => RELU => BatchNormalization => Dropout
Классификатор softmax
Реализация данной функции приведена ниже

Если не понимаете, что тут происходит - документация всегда вам поможет https://keras.io/keras_core/guides/getting_started_with_keras_core/

In [7]:
# Простая convolutional neural network. Используется для распознавания изображений
# из исходного набора mnist и изменённого с помощью FGSM
class SimpleCNN:
  @staticmethod
  def build(width, height, depth, classes):
    # Инициализация модели
    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1
    # Первый набор слоёв CONV => RELU => BN
    model.add(Conv2D(32, (3, 3), strides=(2, 2), padding="same", input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    # Второй набор слоёв CONV => RELU => BN
    model.add(Conv2D(64, (3, 3), strides=(2, 2), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    # Набор слоёв FC => RELU
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    # Классификатор softmax
    model.add(Dense(classes))
    model.add(Activation("softmax"))
    # Вернуть построенную модель
    return model

# 3. Создание состязательного изображения
Здесь мы генерируем вредоносное изображение, используя формулу из п.0

In [6]:
# Формирование вредоносного изображения
# model - построенная модель SimpleCNN
# image - входное изображение
# label - истинное/целевое значение (метка класса для image)
# eps - значение величины шума (оно должно быть достаточно большим, чтобы обмануть модель и достаточно маленьким, чтобы оставаться незаметным глазу, базово используем значение 0.1)
def generate_image_adversary(model, image, label, eps):

  # Здесь нужно подать изображение на вход. Используем tf.cast - приводит тензор к новому типу.
  # image = tf.cast(параметр1, параметр2), где параметр1 - изображение, параметр2 - новый тип данных (uint8, uint16, uint32, uint64, int8, int16, int32, int64, float16, float32, float64, complex64, complex128, bfloat16).
  # Нам нужен тип с плавающей точкой, при использовании целочисленного мы потеряем точность
  image = tf.cast(image, float)

  # Вычисление градиентов
  with tf.GradientTape() as tape:
    tape.watch(image)
    # Делаем предсказание о классе (текущее выходное значение), к которому принадлежит изображение, на основе нашей модели SimpleCNN
    pred = model(image)

    # A затем вычисляем функцию потерь
    # Здесь нужно как-то вычислить нашу функцию потерь, например, используя Mean Squared Error (MSE из keras.losses) между label (истинным/целевым значением, меткой класса для image) и предсказанием pred
    loss = MSE(label, pred)

    # Вычисление градиентов loss по отношению к входному изображению (используем tape.gradient(функция потерь, входное изображение))
    gradient = tape.gradient(loss, image)

    signedGrad = tf.sign(gradient)
    # Вычисляем соревновательное изображение
    adversary = (image + (signedGrad * eps)).numpy()
    # Возвращаем соревновательное изображение
    return adversary

# 4. Загружаем датасет и обучаем модель из п.2 на данных из MNIST
Batch_size влияет на некоторые показатели, такие как общее время обучения, время обучения на эпоху, качество модели и тому подобное. Обычно batch_size выбирается как степень двойки в диапазоне от 16 до 512. Но в целом размер от 32 — это эмпирическое правило и хороший первоначальный выбор. Попробуйте разные варианты, посмотрите, что меняется.

Epochs - влияет на точность модели, но также и на время, затрачиваемое на обучение. Попробуйте разные варианты, например, от 5 до 10.

https://www.sabrepc.com/blog/Deep-Learning-and-AI/Epochs-Batch-Size-Iterations https://neurohive.io/ru/osnovy-data-science/jepoha-razmer-batcha-iteracija/

In [ ]:
# Загрузка и обработка mnist
# trainX - тренировочные данные
# trainY - метки для тренировочных данных
# textX - тестовые данные
# textY - метки для тестовых данных
(trainX, trainY), (testX, testY) = mnist.load_data()
trainX = trainX / 255.0
testX = testX / 255.0
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

In [ ]:
# Инициализация модели
from mnist import MNIST
mndata = MNIST('./dir_with_mnist_data_files')
opt = Adam(lr=1e-3)#Adam(learning_rate=1e-3)
model = SimpleCNN.build(width=28, height=28, depth=1, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt,	metrics=["accuracy"])

# Обучаем CNN на MNIST (здесь нужно написать функцию model.fit(тренировочные данные, метки для тренировочных данных,
#                                                              validation_data=(тестовые данные, метки для тестовых данных),
#                                                              batch_size=количество блоков, на которое производится разделение данных,
#                                                              epochs=количество эпох,
#                                                              verbose=1)
#)
# Поэкспериментируйте с количеством эпох, но accuracy не должна быть ниже 0.99
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=10, batch_size=128, verbose=True)

В следующей ячейке необходимо посчитать и вывести метрики accuracy и loss для обученной модели

In [ ]:
# Делаем предсказания на основе исходного набора данных
# В model.evaluate(x=тестовые данные, y=метки для тестовых данных, verbose=0) необходимо дописать параметры. Verbose задает индикацию о процессе выполнения, здесь она нас не интересует
(loss, acc) = model.evaluate(x=testX, y=testY, verbose=False)
print("loss: {:.4f}, acc: {:.4f}".format(loss, acc))

Сохраним обученную модель:

In [ ]:
model.save('/content/sample_data/model.h5')

# 5. Здесь дописывать ничего не надо, остаётся лишь посмотреть на результат работы вашей CNN и вашего генератора состязательных изображений

In [ ]:
# Создаем массивы для записи сгенерированных изображений
adv_images = []
adv_labels = []

# Для 100 случайных объектов в наборе тренировочных данных
for i in np.random.choice(np.arange(0, len(testX)), size=(100,)):
    # Записываем изображение и предсказание
  image = testX[i]
  label = testY[i]
  # Генерируем состязательное изображение на основании текущего и делаем предсказание для него
  adversary = generate_image_adversary(model, image.reshape(1, 28, 28, 1), label, eps=0.1)
  pred = model.predict(adversary)

  adv_images.append(adversary.reshape(28,28,1))
  adv_labels.append(label.reshape(10))
  # Обработка изображений для вывода
  adversary = adversary.reshape((28, 28)) * 255
  adversary = np.clip(adversary, 0, 255).astype("uint8")
  image = image.reshape((28, 28)) * 255
  image = image.astype("uint8")
  image = np.dstack([image] * 3)
  adversary = np.dstack([adversary] * 3)
  image = cv2.resize(image, (96, 96))
  adversary = cv2.resize(adversary, (96, 96))
  # Запись предсказания
  imagePred = label.argmax()
  adversaryPred = pred[0].argmax()
  color = (0, 255, 0)
  # Если предсказание не совпадает с реальным изображением - цвет подписи красный
  if imagePred != adversaryPred:
    color = (0, 0, 255)
  # Рисуем предсказание в углу выходного изображения
  cv2.putText(image, str(imagePred), (2, 25),	cv2.FONT_HERSHEY_SIMPLEX, 0.95, (0, 255, 0), 2)
  cv2.putText(adversary, str(adversaryPred), (2, 25),	cv2.FONT_HERSHEY_SIMPLEX, 0.95, color, 2)
  # Рисуем исходное изображение с предсказанием и состязательное изображение с предсказанием
  output = np.hstack([image, adversary])
  cv2_imshow(output)
    # В Jupyter надо использовать следующее:
    # IPython.display.image(output)

# Делаем предсказания на основе состязательного набора данных
adv_images = np.array(adv_images)
adv_labels = np.array(adv_labels)
(loss, acc) = model.evaluate(x=adv_images, y=adv_labels, verbose=0)
print("loss: {:.4f}, acc: {:.4f}".format(loss, acc))

# Сохраняем данные в файл в формате .npz
np.savez('/content/sample_data/data.npz', array1=adv_images, array2=adv_labels)